In [10]:
import pickle
import numpy as np

def load_filenames(filenames_file_path):
    """
    Load filenames.pickle file and return a list of all file names
    """
    with open(filenames_file_path, 'rb') as f:
        filenames = pickle.load(f, encoding='latin1')
    return filenames

data_dir = "C:/Users/user/Desktop/birds/birds"
test_dir = data_dir + "/test"
test_text_embeddings = test_dir + "/char-CNN-RNN-embeddings.pickle"

text_embeddings = load_filenames(test_text_embeddings)

print(np.array(text_embeddings).shape)



(2933, 10, 1024)


In [20]:
import random

def load_embeddings(embeddings_file_path):
    """
    훈련된 텍스트 임베딩을 불러옴
    """
    with open(embeddings_file_path, 'rb') as f:
        embeddings = pickle.load(f, encoding='latin1')
        embeddings = np.array(embeddings)
        print('embeddings: ', embeddings.shape)
    return embeddings

data_dir = "C:/Users/user/Desktop/birds/birds"
test_dir = data_dir + "/test"
test_text_embeddings = test_dir + "/char-CNN-RNN-embeddings.pickle"

all_embeddings = load_embeddings(test_text_embeddings)

all_embeddings1 = all_embeddings[0, :, :]

print(all_embeddings1.shape)

embedding_ix = random.randint(0, all_embeddings1.shape[0] - 1)
embedding = all_embeddings1[embedding_ix, :]

print(embedding.shape)


embeddings:  (2933, 10, 1024)
(10, 1024)
(1024,)
